In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils import plot_model

In [2]:
dataset = pd.read_csv("obesity.csv")
print("Classes in the dataset:", dataset['NObeyesdad'].unique())

Classes in the dataset: ['Normal_Weight' 'Overweight_Level_I' 'Overweight_Level_II'
 'Obesity_Type_I' 'Insufficient_Weight' 'Obesity_Type_II'
 'Obesity_Type_III']


In [3]:
print(dataset.columns)

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'NObeyesdad'],
      dtype='object')


In [4]:
categories = ['Gender', 'family_history_with_overweight', 'FAVC', 'SMOKE', 'SCC', 'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
for c in categories:
  dummies = pd.get_dummies(dataset[c], prefix=(str(c)+"_"))
  dataset = pd.concat([dataset, dummies], axis=1)
dataset = dataset.drop(columns=categories)

In [5]:
d = ['NObeyesdad__Insufficient_Weight',
       'NObeyesdad__Normal_Weight', 'NObeyesdad__Obesity_Type_I',
       'NObeyesdad__Obesity_Type_II', 'NObeyesdad__Obesity_Type_III',
       'NObeyesdad__Overweight_Level_I', 'NObeyesdad__Overweight_Level_II']
X = dataset.drop(columns=d)
y = dataset[d]

In [6]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X = pd.DataFrame(data = X_rescaled, columns = X.columns)

In [7]:
data_train, data_test, class_train, class_test = train_test_split(X, y, test_size=0.2)
print(data_train.shape)
print(class_train.shape)

(1688, 31)
(1688, 7)


In [8]:
#Neural Network
def createModel():
  model = Sequential()
  model.add(Dense(128, activation='relu', input_dim = 31))
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(7, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
model = createModel()
model.fit(data_train, class_train, epochs=300, batch_size=50)

Epoch 1/300
34/34 [==============================] - 1s 2ms/step - loss: 1.9136 - accuracy: 0.2032
Epoch 2/300
34/34 [==============================] - 0s 2ms/step - loss: 1.7410 - accuracy: 0.3359
Epoch 3/300
34/34 [==============================] - 0s 2ms/step - loss: 1.5532 - accuracy: 0.4259
Epoch 4/300
34/34 [==============================] - 0s 2ms/step - loss: 1.4151 - accuracy: 0.4502
Epoch 5/300
34/34 [==============================] - 0s 2ms/step - loss: 1.3186 - accuracy: 0.5000
Epoch 6/300
34/34 [==============================] - 0s 2ms/step - loss: 1.2571 - accuracy: 0.5071
Epoch 7/300
34/34 [==============================] - 0s 2ms/step - loss: 1.2286 - accuracy: 0.5249
Epoch 8/300
34/34 [==============================] - 0s 2ms/step - loss: 1.1577 - accuracy: 0.5468
Epoch 9/300
34/34 [==============================] - 0s 2ms/step - loss: 1.1202 - accuracy: 0.5586
Epoch 10/300
34/34 [==============================] - 0s 2ms/step - loss: 1.0579 - accuracy: 0.5942
Epoch 11/

In [9]:
# Accuracy and cost after updating
test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
print('Loss:', test_loss)
print('Accuracy:', test_acc)

Loss: 0.13259856402873993
Accuracy: 0.9645389914512634


In [10]:
dataset_copy = pd.read_csv("obesity.csv")

Cross - Validation for the DNN

In [11]:
#Cross Validation for DNN - https://stackoverflow.com/questions/48085182/cross-validation-in-keras
from sklearn.model_selection import StratifiedKFold
def train_evaluate(model, xtrain, ytrain, xtest, ytest):
  model.fit(data_train, class_train, epochs=300, batch_size=50)
  test_loss, test_acc = model.evaluate(data_test, class_test, verbose=0)
  return test_loss, test_acc
kFold = StratifiedKFold(n_splits = 10)
i=0
sum=0
temp = dataset_copy["NObeyesdad"]
for train, test in kFold.split(X,temp):
  print()
  model = None
  model = createModel()
  loss, acc = train_evaluate(model,X.iloc[[train[0]]],y.iloc[[train[0]]],X.iloc[[test[0]]],y.iloc[[test[0]]])
  i+=1
  sum += acc

print("Mean accuracy:", (sum/i))
#Mean accuracy: 0.9595744669437408

Streaming output truncated to the last 5000 lines.
34/34 [==============================] - 0s 3ms/step - loss: 0.1205 - accuracy: 0.9538
Epoch 206/300
34/34 [==============================] - 0s 2ms/step - loss: 0.1155 - accuracy: 0.9591
Epoch 207/300
34/34 [==============================] - 0s 2ms/step - loss: 0.1172 - accuracy: 0.9597
Epoch 208/300
34/34 [==============================] - 0s 2ms/step - loss: 0.1186 - accuracy: 0.9568
Epoch 209/300
34/34 [==============================] - 0s 3ms/step - loss: 0.1082 - accuracy: 0.9597
Epoch 210/300
34/34 [==============================] - 0s 2ms/step - loss: 0.1234 - accuracy: 0.9573
Epoch 211/300
34/34 [==============================] - 0s 3ms/step - loss: 0.1244 - accuracy: 0.9585
Epoch 212/300
34/34 [==============================] - 0s 2ms/step - loss: 0.1003 - accuracy: 0.9650
Epoch 213/300
34/34 [==============================] - 0s 3ms/step - loss: 0.1041 - accuracy: 0.9645
Epoch 214/300
34/34 [==============================] -